https://huggingface.co/blog/how-to-train

In [33]:
import os

In [34]:
!mkdir -p code-tokenizer-RoBERTa

In [35]:
# from pathlib import Path

# from tokenizers import ByteLevelBPETokenizer

# paths = ['test_sent.txt', 'train_sent.txt']

# # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer()

# # Customize training
# tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
#     "<EOL>","<NUM_LIT>","<STR_LIT>"
# ])

# # # Save files to disk
# tokenizer.save_model(".", "code-tokenizer-RoBERTa")

In [36]:
# from tokenizers.implementations import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcessing


# tokenizer = ByteLevelBPETokenizer(
#     "./code-tokenizer-RoBERTa-vocab.json",
#     "./code-tokenizer-RoBERTa-merges.txt",
# )
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )
# tokenizer.enable_truncation(max_length=512)

# print(
#     tokenizer.encode("for i in range")['input_ids']
# )
# # Encoding(num_tokens=7, ...)
# # tokens: ['<s>', 'Mi', 'Ġestas', 'ĠJuli', 'en', '.', '</s>']


In [37]:
# class CodeDataset():
#     def __init__(self, evaluate: bool=False):
#         tokenizer = ByteLevelBPETokenizer(
#             "./code-tokenizer-RoBERTa-vocab.json",
#             "./code-tokenizer-RoBERTa-merges.txt",
#         )
#         tokenizer._tokenizer.post_processor = BertProcessing(
#             ("</s>", tokenizer.token_to_id("</s>")),
#             ("<s>", tokenizer.token_to_id("<s>")),
#         )
#         tokenizer.enable_truncation(max_length=512)
        
#         self.example=[]
        
#         src_files=['test_sent.txt', 'train_sent.txt']
        
#         for src_file in src_files:
#             print("🔥", src_file)
#             lines = src_file.read_text(encoding="utf-8").splitlines()
#             self.examples += [x.ids for x in tokenizer.encode_batch(lines)]
            
#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i):
#         # We’ll pad at the batch level.
#         return torch.tensor(self.examples[i])

In [38]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from pathlib import Path

paths = ['test_sent.txt', 'train_sent.txt']

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>","<eol>","<str_lit>","<num_lit>"
])
#Save the Tokenizer to disk
!mkdir -p "code-toknzr-roberta"
tokenizer.save_model("code-toknzr-roberta")

['code-toknzr-roberta/vocab.json', 'code-toknzr-roberta/merges.txt']

In [39]:
# Create the tokenizer using vocab.json and mrege.txt files
tokenizer_folder = "code-toknzr-roberta"
tokenizer = ByteLevelBPETokenizer(
    os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
)
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

tokenizer.enable_truncation(max_length=512)
# Test the tokenizer
tokenizer.encode("for i in range")
# Show the tokens created
tokenizer.encode("for i in range").tokens

['<s>', 'for', 'Ġi', 'Ġin', 'Ġrange', '</s>']

In [40]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

# Set a configuration for our RoBERTa model
config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize the model from a configuration without pretrained weights
model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


In [42]:
from transformers import RobertaTokenizerFast

new_tokenizer = RobertaTokenizerFast(tokenizer_object=tokenizer)
new_tokenizer.save_pretrained("code-toknzr-roberta")

('code-toknzr-roberta/tokenizer_config.json',
 'code-toknzr-roberta/special_tokens_map.json',
 'code-toknzr-roberta/vocab.json',
 'code-toknzr-roberta/merges.txt',
 'code-toknzr-roberta/added_tokens.json',
 'code-toknzr-roberta/tokenizer.json')

In [44]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained("code-toknzr-roberta", max_len=512)

In [48]:
class CodeDataset():
    def __init__(self, datafile, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.
        self.examples = []
        # For every value in the dataframe 
        with open(datafile,'r') as fp:
            for example in fp:
                # 
                x=tokenizer.encode_plus(example, max_length = 512, truncation=True, padding=True)
                self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])
      
# Create the train and evaluation dataset
train_dataset = CodeDataset("train_sent.txt", tokenizer)
eval_dataset = CodeDataset("test_sent.txt", tokenizer)

In [49]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [51]:
import torch

# Use Multiple GPUs for Training
use_cuda = torch.cuda.is_available()
torch.cuda.empty_cache()
device = torch.device("cuda" if use_cuda else "cpu")
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=[0,1,2,3], dim=0)
if use_cuda:
    model = model.cuda()

In [52]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir='dec4_gpt',
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=3,
    learning_rate=1e-4,
    weight_decay=0.01,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)

# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #prediction_loss_only=True,
)

In [53]:
# Train the model
# trainer.train()

***** Running training *****
  Num examples = 3059992
  Num Epochs = 3
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 17931
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rohangoli (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/home/rgoli/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/rgoli/.local/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/rgoli/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/rgoli/software/venv/tf1_gpu/lib/python3.7/site-packages/transformers/models/roberta/modeling_roberta.py", line 1097, in forward
    return_dict=return_dict,
  File "/home/rgoli/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/rgoli/software/venv/tf1_gpu/lib/python3.7/site-packages/transformers/models/roberta/modeling_roberta.py", line 854, in forward
    return_dict=return_dict,
  File "/home/rgoli/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _cal

In [2]:
from transformers import pipeline
# Create a Fill mask pipeline
fill_mask = pipeline(
    "fill-mask",
    model="saved_dec7_roberta_a",
    tokenizer="code-toknzr-roberta"
)

Some weights of the model checkpoint at saved_dec7_roberta_a were not used when initializing RobertaForMaskedLM: ['module.roberta.encoder.layer.0.attention.self.query.weight', 'module.roberta.encoder.layer.4.output.dense.bias', 'module.lm_head.bias', 'module.roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'module.roberta.encoder.layer.3.output.LayerNorm.weight', 'module.roberta.encoder.layer.3.attention.self.key.weight', 'module.roberta.encoder.layer.4.attention.self.key.weight', 'module.roberta.encoder.layer.5.attention.self.key.bias', 'module.roberta.encoder.layer.1.output.LayerNorm.weight', 'module.roberta.encoder.layer.5.intermediate.dense.bias', 'module.roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'module.roberta.encoder.layer.3.attention.self.query.weight', 'module.roberta.encoder.layer.2.output.LayerNorm.weight', 'module.lm_head.dense.bias', 'module.roberta.encoder.layer.2.attention.self.key.weight', 'module.roberta.encoder.layer.4.attention.output.Layer

In [4]:
fill_mask("for <mask> in range")

[{'sequence': 'for event in range',
  'score': 0.0012233317829668522,
  'token': 3565,
  'token_str': ' event'},
 {'sequence': 'for cuml in range',
  'score': 0.0010677966056391597,
  'token': 6661,
  'token_str': ' cuml'},
 {'sequence': 'for connected in range',
  'score': 0.0009085682104341686,
  'token': 3004,
  'token_str': ' connected'},
 {'sequence': 'forindexes in range',
  'score': 0.0007934276945888996,
  'token': 4248,
  'token_str': 'indexes'},
 {'sequence': 'forburger in range',
  'score': 0.0007660402334295213,
  'token': 4520,
  'token_str': 'burger'}]

In [5]:
fill_mask("for <mask>")

[{'sequence': 'for cuml',
  'score': 0.0012699144426733255,
  'token': 6661,
  'token_str': ' cuml'},
 {'sequence': 'for event',
  'score': 0.0010557093191891909,
  'token': 3565,
  'token_str': ' event'},
 {'sequence': 'for connected',
  'score': 0.0009457896230742335,
  'token': 3004,
  'token_str': ' connected'},
 {'sequence': 'forindexes',
  'score': 0.0008830957231111825,
  'token': 4248,
  'token_str': 'indexes'},
 {'sequence': 'forburger',
  'score': 0.0008391459123231471,
  'token': 4520,
  'token_str': 'burger'}]

In [6]:
fill_mask("import <mask>")

[{'sequence': 'import cuml',
  'score': 0.001225362764671445,
  'token': 6661,
  'token_str': ' cuml'},
 {'sequence': 'import event',
  'score': 0.0010841487674042583,
  'token': 3565,
  'token_str': ' event'},
 {'sequence': 'importindexes',
  'score': 0.001021808828227222,
  'token': 4248,
  'token_str': 'indexes'},
 {'sequence': 'importburger',
  'score': 0.0008810916333459318,
  'token': 4520,
  'token_str': 'burger'},
 {'sequence': 'import connected',
  'score': 0.0007944402750581503,
  'token': 3004,
  'token_str': ' connected'}]

In [9]:
fill_mask("import sys, <mask>")

[{'sequence': 'import sys,gain',
  'score': 0.0012259475188329816,
  'token': 4980,
  'token_str': 'gain'},
 {'sequence': 'import sys,cover',
  'score': 0.0008220626041293144,
  'token': 4996,
  'token_str': 'cover'},
 {'sequence': 'import sys, cuml',
  'score': 0.0008176953997462988,
  'token': 6661,
  'token_str': ' cuml'},
 {'sequence': 'import sys, event',
  'score': 0.0007884774822741747,
  'token': 3565,
  'token_str': ' event'},
 {'sequence': 'import sys, heap',
  'score': 0.000776717730332166,
  'token': 566,
  'token_str': ' heap'}]

In [15]:
fill_mask("import math <mask>")

[{'sequence': 'import mathindexes',
  'score': 0.0009021040168590844,
  'token': 4248,
  'token_str': 'indexes'},
 {'sequence': 'import mathncb',
  'score': 0.0008219612063840032,
  'token': 5814,
  'token_str': 'ncb'},
 {'sequence': 'import math mean',
  'score': 0.0008200139855034649,
  'token': 2441,
  'token_str': ' mean'},
 {'sequence': 'import math bcount',
  'score': 0.0007356998394243419,
  'token': 6329,
  'token_str': ' bcount'},
 {'sequence': 'import mathsetitem',
  'score': 0.0007050809217616916,
  'token': 5212,
  'token_str': 'setitem'}]

In [16]:
fill_mask("from math <mask>")

[{'sequence': 'from mathncb',
  'score': 0.0008835267508402467,
  'token': 5814,
  'token_str': 'ncb'},
 {'sequence': 'from mathindexes',
  'score': 0.0008264688076451421,
  'token': 4248,
  'token_str': 'indexes'},
 {'sequence': 'from math mean',
  'score': 0.0008002515533007681,
  'token': 2441,
  'token_str': ' mean'},
 {'sequence': 'from math bcount',
  'score': 0.0006951779359951615,
  'token': 6329,
  'token_str': ' bcount'},
 {'sequence': 'from mathelations',
  'score': 0.0006611536373384297,
  'token': 6136,
  'token_str': 'elations'}]